In [1]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook
from itertools import zip_longest
import math

In [2]:
data = pd.read_csv('tokens.csv')
data.drop(data[data['tokens'].isna()].index, inplace=True)
data =  data.reset_index(drop=True)

In [228]:
data.tokens[4]

'скраб лицо skinlite обновлять кожа освежать цвет лицо предназначить тщательный очистка лицо удаление загрязнение отмереть клетка поверхность эпидермис стимулировать процесс обновление кожа содержать сферический полировать частичка обеспечивать эффективный очистка травмировать поверхность кожа раздражать экстракт лимон мёд бамбук нормализовать процесс эпителизация повышать тонус упругость эластичность кожа интенсивно увлажнять смягчать использование скраб skinlite кожа выглядеть молодой гладкий свежеть характеристика вес мл артикул sl производитель южный корея товар сертифицировать'

In [3]:
d_t_c = [] # документ => индекс слова => кол-во
n_t = {} # индекс слова => слово 
doc_ind = 0
token_ind = 0
for i in tqdm_notebook(data.tokens):
    description = str(i)
    docunent_tokens = description.split(' ')
    id_cnt = defaultdict(lambda: int())
    for each_token in docunent_tokens:
        if not each_token in n_t:
            n_t[each_token] = token_ind
            token_ind += 1
        id_cnt[n_t[each_token]] += 1
    doc = list(zip_longest([], id_cnt.keys(), id_cnt.values(), fillvalue=doc_ind))
    d_t_c += doc
    doc_ind += 1
print(len(d_t_c), token_ind)

<ipython-input-3-70ff37e5100f>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(data.tokens):



23284255 257735


In [4]:
COLUMN_NAMES = ['doc', 't_ind', 't_cnt']
df_docs = pd.DataFrame(d_t_c, columns=COLUMN_NAMES) 

In [5]:
df_docs.head()

,doc,t_ind,t_cnt
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [6]:
COLUMN_NAMES = ['ind', 'token']
tokens_zip = zip(n_t.values(), n_t.keys())
df_tokens = pd.DataFrame(tokens_zip, columns=COLUMN_NAMES).set_index('ind')

In [7]:
df_tokens.head()

,token
ind,
0,москва
1,год
2,государственный
3,издательство
4,художественный


# TFIDF алгоритм

In [9]:
cnt_D = data.shape[0]
top_t = []
for doc_ind in tqdm_notebook(range(cnt_D)):
    doc = df_docs[df_docs.doc == doc_ind]
    TFIDF = {}
    t_in_d = sum(doc.t_cnt)
    for _, ind, cnt in doc.values:
        TF = cnt / t_in_d
        D_with_t = len(df_docs[df_docs.t_ind == ind])
        IDF = math.log(cnt_D / D_with_t)
        TFIDF[ind] = TF * IDF
    sorted_keys = sorted(TFIDF, key=TFIDF.get, reverse=True)
    top = sorted_keys[:3]
    top_t.append([doc_ind] + list(df_tokens.iloc[top].token))

<ipython-input-9-c6561ab6d7ce>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc_ind in tqdm_notebook(range(cnt_D)):


KeyboardInterrupt: 

In [364]:
COLUMN_NAMES = ['D_ind', 'top1', 'top2', 'top3']
df_new = pd.DataFrame(top_t, columns=COLUMN_NAMES).set_index('D_ind')
df_new

,top1,top2,top3
D_ind,,,
0,граф,монт,кристо
1,ложка,tescoma,прекрасно
2,энергия,стихия,снежный


# TFIDF sklearn

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer()
word_count_vector = cv.fit_transform(data.tokens)

In [12]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(word_count_vector)

In [13]:
tf_idf = pd.DataFrame(X.toarray() ,columns=cv.get_feature_names())
## ОШИБКА 

MemoryError: Unable to allocate 814. GiB for an array with shape (423932, 257649) and data type float64

In [14]:
top3 = []
for doc_ind in tqdm_notebook(range(len(data.tokens))):
    top3.append([doc_ind] + list(tf_idf.iloc[doc_ind].sort_values(ascending=[False])[:3].keys()))

<ipython-input-14-c97f4bbc7c7f>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc_ind in tqdm_notebook(range(len(data.tokens))):


NameError: name 'tf_idf' is not defined

In [365]:
COLUMN_NAMES = ['D_ind', 'top1', 'top2', 'top3']
df_new2 = pd.DataFrame(top3, columns=COLUMN_NAMES).set_index('D_ind')
df_new2

,top1,top2,top3
D_ind,,,
0,кристо,граф,монт
1,ложка,нержавеющий,tescoma
2,снежный,лавина,стихия
